# 머신러닝 비교 
- 학습: 현대쇼핑 / 쿠팡 쌀 리뷰 ( 긍정 +1 / 부정 0)

- 예측: sns데이터 ( 2015 카페 )

In [1]:
# 패키지 설치
import pandas as pd
#warning 메시지 표시 안함
import warnings
warnings.filterwarnings(action = 'ignore')
from konlpy.tag import Okt 
okt = Okt()

## 01. 데이터 불러오기

### - 일반 현대 쇼핑 train set

In [2]:
# Train 데이터 불러오기
train_df = pd.read_excel('train_data/naver_steam_hyundai_shop.xlsx',header=None)

In [3]:
train_df[0:3] # 데이터 확인

,0,1
0,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...,0
1,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...,0
2,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...,1


### - 쿠팡 쌀 관련 train set

In [4]:
train_df_2 = pd.read_excel('train_data/coupang_train.xlsx') # 데이터 불러오기
train_df_2[0:3] # 확인

,제품,내용,긍부정,원별점
0,메뚜기,2021년 2월 15일 최근 도정입니다.\n2월 26일 배송 받았고 처음 밥해 먹었...,0,1
1,메뚜기,쌀 반정도 부었는데\n돌맹이 나왔어요 ㅡㅡ::\n손톱으로 누르니 돌이네요..\n왜 ...,0,1
2,메뚜기,아 정말 사본쌀중에 제일 별로였어요\n저는 31280원 주고 구매했는데 싸지도않도 ...,0,1


### - test set 

In [7]:
# Test 데이터 불러오기
pred_df = pd.read_excel('test_data/0902_2015_cafe.xlsx') # header있음 # 전처리 완료된 데이터 문서단위임

In [9]:
# 데이터 확인
pred_df['content']

0       세종시로 올해 1월 10일 이사했습니다||-|| 어제 쌀이 똑 떨어졌는데 마침 까페...
1       저희 큰아버지께서 쌀도매를 하세요||-||  빨간쌀 이예요||-|| 저도 작녁부터 ...
2                                     제가 변비가 심해서 현미쌀100 를
3       그동안 계속 공구후기며 공구게시판에서 보던 쌀찐빵을 구입했어요||-|| 아기간식도 ...
4       심심한 일요일 오후 당근쌀머핀 만들어 보았어요||-|| 당근이랑 우유는 휘리릭 갈아...
                              ...                        
9581    요리전용맛술 마트가면 보기는 했는데 저게 꼭 필요할까 하는 생각에 구매를 망설였는데...
9582                           가능한가요     믹서기 청소가 귀찮아서    
9583    담주부터 초기이유식 시작인데 아직 애가 못앉잖아요  다들 그냥 분유주듯 안아서 먹이...
9584                 어뭉들어디서 사드시나요  아이간식사먹이는 사이트추천좀해주셔요   
9585    간혹 동물 사료용으로 쓰신다고 무료나눔하신던데   벌레가 막 나오거나 하지는 않는데...
Name: content, Length: 9586, dtype: object

## 02. 데이터 전처리_결측값 처리

## - TRAIN SET 전처리

 - 쇼핑 학습데이터

In [10]:
# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
train_df = train_df[train_df[0].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(train_df.info())

# 분류 클래스의 구성을 확인
print(train_df[1].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356832 entries, 0 to 356831
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       356832 non-null  object
 1   1       356832 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.2+ MB
None
1    179180
0    177652
Name: 1, dtype: int64


 - 쿠팡 학습데이터

In [11]:
train_df_2 ['내용'] # train_x에 추가

0       2021년 2월 15일 최근 도정입니다.\n2월 26일 배송 받았고 처음 밥해 먹었...
1       쌀 반정도 부었는데\n돌맹이 나왔어요 ㅡㅡ::\n손톱으로 누르니 돌이네요..\n왜 ...
2       아 정말 사본쌀중에 제일 별로였어요\n저는 31280원 주고 구매했는데 싸지도않도 ...
3       아주 그냥 밥 할때마다 열받아서...\n오늘도 저녁밥 짓기전에 쌀을보니... 아무래...
4       최악이요!!자주 시켜먹었는데 2020년 쌀이 새로와서 시켰는데 정말 맛도 없어요!!...
                              ...                        
6780                      매번 재구매하고 있어요\n찰지게 밥도 잘 되고 맛있습니다
6781    곰곰에서 쌀도 나오다니 ㅎㅎㅎ 10킬로짜리 살 일이 없었는데 최근에 집에 있는 시간...
6782                20kg 없어서 아쉬워요!!!\n쌀 좋아요!! 곰곰쌀만 먹고있어요!
6783    요즘 쌀 20 키로 에 비싼 것은 육만 원 가량 하고 저렴한 것은 사만원대 정도하는...
6784    매일 그날 도정한 쌀 사다가 먹었는데..이번에 구매하려니 품절이라 곰곰 쌀을 구매해...
Name: 내용, Length: 6785, dtype: object

In [12]:
train_df_2 ['긍부정'] # train_y에 추가

0       0
1       0
2       0
3       0
4       0
       ..
6780    1
6781    1
6782    1
6783    1
6784    1
Name: 긍부정, Length: 6785, dtype: int64

In [13]:
train_df_2.isnull().sum()

제품     0
내용     0
긍부정    0
원별점    0
dtype: int64

## - train set 합치기

In [14]:
len (train_df[0]), len(train_df_2['내용'])

(356832, 6785)

In [15]:
# train_x 합치기
all_train_x_df= pd.concat( [train_df[0], train_df_2['내용'] ], axis = 0)
all_train_x_df=all_train_x_df.reset_index()
all_train_x_df

,index,0
0,0,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
1,1,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
2,2,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
3,3,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
4,4,+@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
...,...,...
363612,6780,매번 재구매하고 있어요\n찰지게 밥도 잘 되고 맛있습니다
363613,6781,곰곰에서 쌀도 나오다니 ㅎㅎㅎ 10킬로짜리 살 일이 없었는데 최근에 집에 있는 시간...
363614,6782,20kg 없어서 아쉬워요!!!\n쌀 좋아요!! 곰곰쌀만 먹고있어요!
363615,6783,요즘 쌀 20 키로 에 비싼 것은 육만 원 가량 하고 저렴한 것은 사만원대 정도하는...


In [16]:
# train_y합치기
all_train_y_df= pd.concat( [train_df[1], train_df_2['긍부정'] ], axis = 0)
all_train_y_df=all_train_y_df.reset_index()
all_train_y_df

,index,0
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
363612,6780,1
363613,6781,1
363614,6782,1
363615,6783,1


In [17]:
all_train_y_df[0][1] #확인

0

## - pred set 전처리

 - 문서 단위 pred   : pred_df['content']

In [21]:
# pred에 str이 아닌게 존재하네! nan임을 확인함 # 제거하고 데이터 다시 가져옴
for i in range(len(pred_df)):
    if type(pred_df['content'][i]) != str :
        print(i)

In [22]:
pred_df['content'].isnull().sum() # null값 없음

0

In [23]:
# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
pred_df = pred_df[pred_df['content'].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(pred_df.info())

# null값 개수 확인
print(pred_df['content'].isnull().sum())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9586 entries, 0 to 9585
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    9586 non-null   int64 
 1   Unnamed: 0.1  9586 non-null   int64 
 2   title         9586 non-null   object
 3   nickname      9586 non-null   object
 4   date          9586 non-null   object
 5   content       9586 non-null   object
 6   comments      8846 non-null   object
dtypes: int64(2), object(5)
memory usage: 599.1+ KB
None
0


In [24]:
pred_df['content'].reset_index

<bound method Series.reset_index of 0       세종시로 올해 1월 10일 이사했습니다||-|| 어제 쌀이 똑 떨어졌는데 마침 까페...
1       저희 큰아버지께서 쌀도매를 하세요||-||  빨간쌀 이예요||-|| 저도 작녁부터 ...
2                                     제가 변비가 심해서 현미쌀100 를
3       그동안 계속 공구후기며 공구게시판에서 보던 쌀찐빵을 구입했어요||-|| 아기간식도 ...
4       심심한 일요일 오후 당근쌀머핀 만들어 보았어요||-|| 당근이랑 우유는 휘리릭 갈아...
                              ...                        
9581    요리전용맛술 마트가면 보기는 했는데 저게 꼭 필요할까 하는 생각에 구매를 망설였는데...
9582                           가능한가요     믹서기 청소가 귀찮아서    
9583    담주부터 초기이유식 시작인데 아직 애가 못앉잖아요  다들 그냥 분유주듯 안아서 먹이...
9584                 어뭉들어디서 사드시나요  아이간식사먹이는 사이트추천좀해주셔요   
9585    간혹 동물 사료용으로 쓰신다고 무료나눔하신던데   벌레가 막 나오거나 하지는 않는데...
Name: content, Length: 9586, dtype: object>

In [25]:
pred_df['content']

0       세종시로 올해 1월 10일 이사했습니다||-|| 어제 쌀이 똑 떨어졌는데 마침 까페...
1       저희 큰아버지께서 쌀도매를 하세요||-||  빨간쌀 이예요||-|| 저도 작녁부터 ...
2                                     제가 변비가 심해서 현미쌀100 를
3       그동안 계속 공구후기며 공구게시판에서 보던 쌀찐빵을 구입했어요||-|| 아기간식도 ...
4       심심한 일요일 오후 당근쌀머핀 만들어 보았어요||-|| 당근이랑 우유는 휘리릭 갈아...
                              ...                        
9581    요리전용맛술 마트가면 보기는 했는데 저게 꼭 필요할까 하는 생각에 구매를 망설였는데...
9582                           가능한가요     믹서기 청소가 귀찮아서    
9583    담주부터 초기이유식 시작인데 아직 애가 못앉잖아요  다들 그냥 분유주듯 안아서 먹이...
9584                 어뭉들어디서 사드시나요  아이간식사먹이는 사이트추천좀해주셔요   
9585    간혹 동물 사료용으로 쓰신다고 무료나눔하신던데   벌레가 막 나오거나 하지는 않는데...
Name: content, Length: 9586, dtype: object

- 문장 단위 pred :data_content['content']

    문서 -> 문장 변환

In [26]:
# contents str로 처리
q= " "
for i in range(len(pred_df)):
    try:
     q += pred_df['content'][i]
    except:
     pass

In [27]:
# 문장어미 구분자 대치
q = q.replace('네요','네요||-||').replace('하죠','하죠||-||').replace('데요','데요||-||').replace('이네요','이네요||-||')
q = q.replace('네요.','네요||-||').replace('되죠','되죠||-||').replace('대요','대요||-||').replace('이요','이요||-||')
q = q.replace('돼요.','돼요||-||').replace('되요','되요||-||').replace('다닙니다','다닙니다||-||').replace('입니다','입니다||-||')
q = q.replace('입니다.','입니다||-||').replace('니다','니다||-||').replace('중입니다','중입니다||-||').replace('입니다','입니다||-||')
q = q.replace('나요.','나요||-||').replace('어요','어요||-||').replace('까요','까요||-||').replace('에요','에요||-||')
q = q.replace('이에요','이에요||-||').replace('봐요','봐요||-||').replace('아요.','아요||-||').replace('와요','와요||-||')
q = q.replace('해요','해요||-||').replace('려요','려요||-||').replace('구요','구요||-||').replace('고요','고요||-||')
q = q.replace('죠','죠||-||').replace(':)','').replace('더라','더라||-||').replace('게요','게요||-||').replace('께요','께요||-||').replace('세요','세요||-||').replace('든요','든요||-||').replace('예요','예요||-||').replace('서요','서요||-||')
q = q.replace('않았다','않았다||-||').replace('었다','었다||-||').replace('있다','있다||-||').replace('이다','이다||-||').replace('싶다','싶다||-||').replace('않다','않다||-||').replace('되었다','되었다||-||').replace('아니다','아니다||-||')
q = q.replace('하자','하자||-||').replace('는다','는다||-||').replace('했다','했다||-||').replace('한다','한다||-||').replace('혔다','혔다||-||').replace('난다','난다||-||').replace('같아요','같아요||-||').replace('되었어','되었어||-||').replace('했어','했어||-||').replace('했지','했지||-||').replace('었어','었어||-||')

In [28]:
#특수문자 제거
q = q.replace('.','').replace('?','').replace('~','').replace('!','').replace('^','').replace('#','').replace('★','').replace('ㅋ','').replace('ㅜ','').replace('ㅠ','').replace('ㅎ','').replace('ㅏ','').replace('ㅡ','')

In [29]:
# 구분자로 문장 분리
p = q.split('||-||')
len(p), len(q)

(88910, 2099604)

In [30]:
# 각 문장을 데이터 프레임으로 저장
sentence_Data = pd.DataFrame({'content': p})
len(sentence_Data)

88910

In [31]:
sentence_Data # pred할 셋 문장단위 설정 완

,content
0,세종시로 올해 1월 10일 이사했습니다
1,
2,어제 쌀이 똑 떨어졌는데 마침 까페에 쌀공구가 떴네요
3,
4,내일아침 밥을 하기위해 저녁에 직접 방문드리기로 했습니다
...,...
88905,멸치도 생선이니 다음에는 캠핑장에가서 고기 구워먹을때 사용해 봐야 겠여요가능...
88906,
88907,큰봉지로 한봉지정도되요
88908,


    문장단위 데이터 전처리

In [32]:
# 2번 이상 중복인 데이터 확인
sentence = sentence_Data['content'].value_counts()
idx_sentence=sentence[sentence >= 2].index.tolist()
len(idx_sentence), idx_sentence

(495,
 ['',
  '요',
  '구요',
  '고요',
  '     공유소스 코드                                   주소복사     소스544   306740   416936   5261280   720복사      미지원으로        코드 퍼가기 기능은 더 이상 지원되지않습니다',
  '360       도움말            멀티트랙의 트랙별 영상은 우측 관련영상에서 볼 수 있습니다',
  ' 5분 이상 소요될 수 있으며  영상의 길이에 따라 소요 시간은 달라집니다',
  '네요',
  ' 감사합니다',
  '  안녕하세요',
  '고 합니다',
  ' 안녕하세요',
  '  감사합니다',
  '   감사합니다',
  '고 하네요',
  '고 하더라',
  '    안녕하세요',
  '안녕하세요',
  '  이전 질문 글 확인 후 꼭 작성해주세요',
  '   안녕하세요',
  '     타인의 사진을 이용한 질문은 금지합니다',
  '고 해요',
  '     안녕하세요',
  ' 도와주세요',
  '  알려주세요',
  '감사합니다',
  '                               639756    카페 규정 어길시 무통보삭제 및 무통보 활동정지 처리 됩니다',
  ' 2  수다방에는 판매글 및 카페규정에 어긋나는 글 올릴시 삭제처리됩니다',
  '  도와주세요',
  '  1  카페 규정 확인 후 글 남겨주세요',
  '   푸짐한 속재료  깔끔한 데코 덧글달아 주시고 쪽지보내주세요',
  '죠',
  '   질문이 아닌글을 남기시면 게시글 이동요청후 글이 삭제됩니다',
  ' 알려주세요',
  '   알려주세요',
  '   3단계 등업 시 질문글을 포함되지 않으며 질문에 대한 답변글은 포함됩니다',
  '   성실멤버 등업 시 질문글을 포함되지 않으며 질문에 대한 답변글은 포함됩니다',
  '  양해 부탁드립니다',
  ' 또한 덧글로             등  단순 기호나 10자 미만 덧글

In [33]:
# 중복제거
data2 = sentence_Data
for i in idx_sentence:
    data2 = data2[sentence_Data['content'] != i] # 중복 완료해서 저장

In [34]:
len(sentence_Data), len(data2) #중복제거 전 데이터, 중복제거완료 데이터

(88910, 36354)

In [35]:
data2.isnull().sum()

content    0
dtype: int64

In [36]:
data2

,content
0,세종시로 올해 1월 10일 이사했습니다
2,어제 쌀이 똑 떨어졌는데 마침 까페에 쌀공구가 떴네요
4,내일아침 밥을 하기위해 저녁에 직접 방문드리기로 했습니다
6,세종청사에서 출발해 15분 걸린듯 싶어요
8,깜깜한 밤이라 아주 약간 길을 헤맸는데 큰길옆에 소나무가 있는집 이라 다음번엔 금...
...,...
88899,아이들 밑반찬으로 짱인 멸치볶음 멸치에 남아 있는 비린내 잡기 딱 좋더라
88903,식용유 미림맛술 꿀 이렇게 3가지 양념만 넣고 조리 했는데 맛이 와 비린맛 잘 잡...
88905,멸치도 생선이니 다음에는 캠핑장에가서 고기 구워먹을때 사용해 봐야 겠여요가능...
88907,큰봉지로 한봉지정도되요


In [37]:
data_content = data2[['content']].dropna(axis=0)
data_content.reset_index(drop=True, inplace=True)
data_content

,content
0,세종시로 올해 1월 10일 이사했습니다
1,어제 쌀이 똑 떨어졌는데 마침 까페에 쌀공구가 떴네요
2,내일아침 밥을 하기위해 저녁에 직접 방문드리기로 했습니다
3,세종청사에서 출발해 15분 걸린듯 싶어요
4,깜깜한 밤이라 아주 약간 길을 헤맸는데 큰길옆에 소나무가 있는집 이라 다음번엔 금...
...,...
36349,아이들 밑반찬으로 짱인 멸치볶음 멸치에 남아 있는 비린내 잡기 딱 좋더라
36350,식용유 미림맛술 꿀 이렇게 3가지 양념만 넣고 조리 했는데 맛이 와 비린맛 잘 잡...
36351,멸치도 생선이니 다음에는 캠핑장에가서 고기 구워먹을때 사용해 봐야 겠여요가능...
36352,큰봉지로 한봉지정도되요


In [38]:
# pred에 str이 아닌게 존재하네! nan임을 확인함 # 제거하고 데이터 다시 가져옴
for i in range(len(data2)):
    if type(data_content['content'][i]) != str :
        print(i) # null없음 확인

## 03. train / test 셋 설정

In [39]:
all_train_x_df[0]

0          +@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
1          +@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
2          +@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
3          +@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
4          +@음침한 분위기+@기괴한 크리쳐들-@눈만 아픈 그래픽-@왜 넣었는지 모를 레터박...
                                ...                        
363612                      매번 재구매하고 있어요\n찰지게 밥도 잘 되고 맛있습니다
363613    곰곰에서 쌀도 나오다니 ㅎㅎㅎ 10킬로짜리 살 일이 없었는데 최근에 집에 있는 시간...
363614                20kg 없어서 아쉬워요!!!\n쌀 좋아요!! 곰곰쌀만 먹고있어요!
363615    요즘 쌀 20 키로 에 비싼 것은 육만 원 가량 하고 저렴한 것은 사만원대 정도하는...
363616    매일 그날 도정한 쌀 사다가 먹었는데..이번에 구매하려니 품절이라 곰곰 쌀을 구매해...
Name: 0, Length: 363617, dtype: object

In [40]:
# Train용 데이터셋과 Test용 데이터 셋 분리
# 1. 예측력을 높이기 위해 수집된 데이터를 학습용과 테스트 용으로 분리하여 진행
# 2. 보통 20~30%를 테스트용으로 분리해 두고 테스트

from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(train_df[0], train_df[1] , test_size=0.2, random_state=0)  # 현대만 학습
train_x, test_x, train_y, test_y = train_test_split(all_train_x_df[0], all_train_y_df[0] , test_size=0.2, random_state=0) # 현대 쿠팡학습

print(len(train_x), len(train_y), len(test_x), len(test_y))

290893 290893 72724 72724


## 04. 토큰화 및 TF-IDF 벡터화

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

  (0, 148569)	0.47549836492869546
  (0, 131056)	0.4236714570780322
  (0, 82234)	0.2958985916358402
  (0, 61807)	0.4687309199118689
  (0, 61752)	0.27660664127719503
  (0, 11956)	0.36228504952231233
  (0, 11642)	0.28174005481645437
  (1, 274307)	0.08145756628760677
  (1, 273064)	0.20903522885535022
  (1, 272493)	0.08514052535169798
  (1, 270011)	0.21628201100834926
  (1, 270010)	0.2020164323997035
  (1, 269913)	0.16694685392740186
  (1, 260953)	0.139527533999439
  (1, 260886)	0.10005091061380157
  (1, 253268)	0.22502365763453194
  (1, 253260)	0.1333262115586509
  (1, 248111)	0.16402436846522514
  (1, 248033)	0.0930755117332171
  (1, 242652)	0.1881946337817182
  (1, 242651)	0.15587049379577164
  (1, 203183)	0.17766753546778244
  (1, 200187)	0.040631807130976916
  (1, 194481)	0.1338393835673231
  (1, 193237)	0.06019768758225192
  :	:
  (290890, 33069)	0.19244959995110983
  (290890, 17890)	0.27514216129565755
  (290890, 8962)	0.12715121731835666
  (290891, 234730)	0.3088098984879182
  (2908

## 05. 감성 분석 모델 구축

In [42]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [3,5,8, 11, 13,]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3} 0.8673876584364476


## 06. 분석 모델 평가

In [43]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)
from sklearn.metrics import accuracy_score
print('감성 분류 모델의 정확도 : ',round(accuracy_score(test_y, test_predict), 3))

감성 분류 모델의 정확도 :  0.871


# 감성 평가 pred

### - 예시 확인

In [44]:
import re
input_text = '딱히 대단한 재미도 감동도 없는데 ~! 너무 과대 평과된 영화 중 하나'
#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = [" ".join(input_text)]
# 입력 텍스트의 피처 벡터화
st_tfidf = tfv.transform(input_text)

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = grid_cv.best_estimator_.predict(st_tfidf)

#예측 결과 출력
if(st_predict == 0):
    print(input_text)
    print('예측 결과: ->> 부정 감성')
else :
    print(input_text)
    print('예측 결과: ->> 긍정 감성')

['딱히 대단한 재미도 감동도 없는데 너무 과대 평과된 영화 중 하나']
예측 결과: ->> 부정 감성


### - 문서 단위 긍부정 평가

In [45]:
input_text_list = pred_df['content']

In [46]:
input_text,st_tfidf,st_predict,result  = [],[],[],[]
#입력 텍스트에 대한 전처리 수행
for i in range(len(input_text_list)):
    input_text_ex = ( re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text_list[i]) )
    input_text.append( [" ".join(input_text_ex) ] )# 한문장 정리
# 입력 텍스트의 피처 벡터화
    st_tfidf.append(tfv.transform(input_text[i])  ) 

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
    st_predict.append (grid_cv.best_estimator_.predict(st_tfidf[i]) )

# 결과 출력  
    if(st_predict[i] == 0):
        result.append('부정')
    else :
        result.append('긍정')

In [47]:
result_df = pd.DataFrame({'sentence': input_text_list,
                         'score':st_predict,
                        'result':result })
result_df

,sentence,score,result
0,세종시로 올해 1월 10일 이사했습니다||-|| 어제 쌀이 똑 떨어졌는데 마침 까페...,[1],긍정
1,저희 큰아버지께서 쌀도매를 하세요||-|| 빨간쌀 이예요||-|| 저도 작녁부터 ...,[1],긍정
2,제가 변비가 심해서 현미쌀100 를,[0],부정
3,그동안 계속 공구후기며 공구게시판에서 보던 쌀찐빵을 구입했어요||-|| 아기간식도 ...,[1],긍정
4,심심한 일요일 오후 당근쌀머핀 만들어 보았어요||-|| 당근이랑 우유는 휘리릭 갈아...,[0],부정
...,...,...,...
9581,요리전용맛술 마트가면 보기는 했는데 저게 꼭 필요할까 하는 생각에 구매를 망설였는데...,[1],긍정
9582,가능한가요 믹서기 청소가 귀찮아서,[0],부정
9583,담주부터 초기이유식 시작인데 아직 애가 못앉잖아요 다들 그냥 분유주듯 안아서 먹이...,[0],부정
9584,어뭉들어디서 사드시나요 아이간식사먹이는 사이트추천좀해주셔요,[1],긍정


In [48]:
result_df.to_excel('result/2015_cafe_contents_senti_ml_pluscoupang.xlsx',encoding='utf-8')

### - 문장단위 긍부정 평가

In [49]:
input_text_list = data_content['content']

In [50]:
input_text,st_tfidf,st_predict,result  = [],[],[],[]
#입력 텍스트에 대한 전처리 수행
for i in range(len(input_text_list)):
    input_text_ex = ( re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text_list[i]) )
    input_text.append( [" ".join(input_text_ex) ] )# 한문장 정리
# 입력 텍스트의 피처 벡터화
    st_tfidf.append(tfv.transform(input_text[i])  ) 

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
    st_predict.append (grid_cv.best_estimator_.predict(st_tfidf[i]) )

# 결과 출력  
    if(st_predict[i] == 0):
        result.append('부정')
    else :
        result.append('긍정')

In [51]:
result_df = pd.DataFrame({'sentence': input_text_list,
                         'score':st_predict,
                        'result':result })
result_df

,sentence,score,result
0,세종시로 올해 1월 10일 이사했습니다,[1],긍정
1,어제 쌀이 똑 떨어졌는데 마침 까페에 쌀공구가 떴네요,[0],부정
2,내일아침 밥을 하기위해 저녁에 직접 방문드리기로 했습니다,[1],긍정
3,세종청사에서 출발해 15분 걸린듯 싶어요,[1],긍정
4,깜깜한 밤이라 아주 약간 길을 헤맸는데 큰길옆에 소나무가 있는집 이라 다음번엔 금...,[1],긍정
...,...,...,...
36349,아이들 밑반찬으로 짱인 멸치볶음 멸치에 남아 있는 비린내 잡기 딱 좋더라,[1],긍정
36350,식용유 미림맛술 꿀 이렇게 3가지 양념만 넣고 조리 했는데 맛이 와 비린맛 잘 잡...,[1],긍정
36351,멸치도 생선이니 다음에는 캠핑장에가서 고기 구워먹을때 사용해 봐야 겠여요가능...,[1],긍정
36352,큰봉지로 한봉지정도되요,[0],부정


In [52]:
result_df.to_excel('result/2015_cafe_sentence_senti_ml_pluscoupang.xlsx',encoding='utf-8')